In [1]:
# 多进程版，这里不能显示出运行结果，但在pycharm里可以正常运行并显示出结果
import requests, re
import selenium
from selenium import webdriver
import lxml, lxml.etree
from bs4 import BeautifulSoup
import chardet
import multiprocessing
from multiprocessing import Manager
import os

def getnumber(url):
    pagetxt = requests.get(url).content #text返回unicode，content，str，text返回unicode，content返回str
    myxml = lxml.etree.HTML(pagetxt.decode('GB2312',errors="ignore"))
    mylist = myxml.xpath("//*[@class=\"pagination\"]/text()")
    text = mylist[len(mylist)-1].strip()
    pat = re.compile("\d+", re.IGNORECASE)
    datalist = pat.findall(text)
    return eval(datalist[0])

def makeurllist(numbers):
    urllist = []
    if numbers%30 == 0:
        for i in range(numbers//30):
            urllist.append("http://wz.sun0769.com/index.php/question/questionType?type=4&page="+str(i*30))
    else:
        for i in range(numbers//30+1):
            urllist.append("http://wz.sun0769.com/index.php/question/questionType?type=4&page="+str(i*30))
    return urllist

def getdatafromurl(urllist, queue):
    for url in urllist:
        try:
            pagetxt = requests.get(url).content #text返回unicode，content str，
            myxml = lxml.etree.HTML(pagetxt.decode("GB2312", errors="ignore")) #编码
            mytable = myxml.xpath("//*[@cellpadding=\"0\"]//*[@cellpadding=\"1\"]")
            idlist = []
            typelist = []
            titlelist = []
            aboutlist = []
            statuslist = []
            namelist = []
            datelist = []
            for line in mytable:
                idlist = line.xpath("//td[1]/text()")
                typelist = line.xpath("//td[2]/a[1]/text()")
                titlelist = line.xpath("//td[2]/a[2]/text()")
                aboutlist = line.xpath("//td[2]/a[3]/text()")
                statuslist = line.xpath("//td[3]/span/text()")
                namelist = line.xpath("//td[4]/text()")
                datelist = line.xpath("//td[5]/text()")
                for i in range(len(typelist)):
                    mygetstr = ""
                    mygetstr += idlist[i+1]
                    mygetstr += " # "
                    mygetstr += typelist[i]
                    mygetstr += " # "
                    mygetstr += titlelist[i]
                    mygetstr += " # "
                    mygetstr += aboutlist[i]
                    mygetstr += " # "
                    mygetstr += statuslist[i]
                    mygetstr += " # "
                    mygetstr += namelist[i+1]
                    mygetstr += " # "
                    mygetstr += datelist[i+1]
                    mygetstr += "\r\n"
                queue.put(mygetstr) #压入数据
                for i in range(len(typelist)):
                    site = {
                        "idlist": idlist[i+1],
                        "typelist": typelist[i],
                        "titlelist": titlelist[i],
                        "aboutlist": aboutlist[i],
                        "statuslist": statuslist[i],
                        "namelist": namelist[i+1],
                        "datelist": datelist[i+1]
                    }
                    print("{idlist} {typelist} {titlelist} {aboutlist} {statuslist} {namelist} {datelist}" .format(**site))
        except:
            print("error")
    print("%s exit" %os.getpid())

if __name__ == "__main__":
#     queue = multiprocessing.Manager.Queue() #进程之间传递数据
    queue = Manager().Queue()
    pagenum = getnumber("http://wz.sun0769.com/index.php/question/questionType?type=4&page=400")
    urllist = makeurllist(pagenum)
    urllist = urllist[:10]
    xclist = [[],[],[]]
    N = len(xclist)
    for i in range(len(urllist)):
        xclist[i%N].append(urllist[i]) #求模切割
    processlist = [] #进程列表
    for i in range(N):
        process = multiprocessing.Process(target=getdatafromurl, args=(xclist[i], queue))
        process.start()
        processlist.append(process)
    print('......start')
    for p in processlist:
        p.join() #等待所有进程退出
    print("......ok")
    while not queue.empty():
        data = queue.get()
        print(data)

......start
......ok


In [2]:
# coding:utf-8
# 多线程测试，函数里面print没有显示出来，但在terminal和IDE里面可以显示
import multiprocessing
import time
from multiprocessing import Manager, Process

def go(N, name, queue):
    time.sleep(1)
    for i in range(N):
        queue.put(name + str(i))  # 进程压入数据

if __name__ == '__main__':
#     queue = multiprocessing.Queue()  # 进程之间传递数据
    queue = Manager().Queue()
    processlist = []

    for name in ["A", "B", "C", "E", "F", "G"]:
        process = Process(target=go, args=(50, name, queue))
        process.start()
        processlist.append(process)  # 开启多个进程

    for p in processlist:
        p.join()  # 等待所有进程退出

    while not queue.empty():
        data = queue.get()
        print("**********")
        print("get %s" %data)

In [3]:
# 进程测试
import multiprocessing
import os
def run_proc(name):
    print('Child process {0} {1} Running '.format(name, os.getpid()))
if __name__ == '__main__':
    print('Parent process {0} is Running'.format(os.getpid()))
    for i in range(5):
        p = multiprocessing.Process(target=run_proc, args=(str(i),))
        print('process start')
        p.start()
    p.join()
    print('Process close')

Parent process 11928 is Running
process start
process start
process start
process start
process start
Process close


In [4]:
# 进程测试
import multiprocessing as mp

def job(a,d):
    print('.....%s aaaaa %s' %(a, d))
if __name__=='__main__':
    p1 = mp.Process(target=job,args=(1,2))
    p1.start()
    p1.join()
job(2, 3)

.....2 aaaaa 3


In [5]:
# coding:utf-8
# 在IDE比如pycharm里可以正常运行并显示，jupyter里不能显示
import multiprocessing
import time
import requests
import lxml
import re
import lxml.etree
import os
#  输入url,返回一个数据集合list
def  download(url):
    pagetxt = requests.get(url).content  # text返回unicode ,content  str
    # print pagetxt.decode("GB2312",errors="ignore") #必须解码
    myxml = lxml.etree.HTML(pagetxt.decode("GB2312", errors="ignore"))  # 编码
    # 浏览器xpath  mytable=myxml.xpath("//*[@id=\"morelist\"]/div/table[2]/tbody/tr/td/table")
    mytable = myxml.xpath("//*[@cellpadding=\"0\"]//*[@cellpadding=\"1\"]")
    idlist = []
    typelist = []
    titlelist = []
    aboutlist = []
    statuslist = []
    namelist = []
    datalist = []
    for line in mytable:
        idlist = line.xpath("//td[1]/text()")
        typelist = line.xpath("//td[2]/a[1]/text()")
        titlelist = line.xpath("//td[2]/a[2]/text()")
        aboutlist = line.xpath("//td[2]/a[3]/text()")
        statuslist = line.xpath("//td[3]/span/text()")
        namelist = line.xpath("//td[4]/text()")
        datelist = line.xpath("//td[5]/text()")

        for i in range(len(typelist)):
            #print idlist[i + 1], typelist[i], titlelist[i], aboutlist[i], statuslist[i], namelist[i + 1], datelist[
                #i + 1]
            mygetstr = ""
            mygetstr +=idlist[i + 1]
            mygetstr += " # "
            mygetstr += typelist[i]
            mygetstr += " # "
            mygetstr +=titlelist[i]
            mygetstr += " # "
            mygetstr += aboutlist[i]
            mygetstr += " # "
            mygetstr += statuslist[i]
            mygetstr += " # "
            mygetstr += namelist[i+1]
            mygetstr += " # "
            mygetstr += datelist[i+1]
            mygetstr+="\r\n"  #换行
            datalist.append(mygetstr)
    return  datalist

def go(url, queue):
    linelist=download(url)
    print("%s get data" %os.getpid())
    for line in linelist:
        queue.put(line)  # 进程压入数据
    print("%s pushed data" %os.getpid())

if __name__ == "__main__":
    #queue = multiprocessing.Queue()  # 进程之间传递数据,python2不可以，python3,
    #  error
    queue= multiprocessing.Manager().Queue() #多进程
    processlist = []
    for url in  ["http://wz.sun0769.com/index.php/question/questionType?type=4&page=81240",
                 "http://wz.sun0769.com/index.php/question/questionType?type=4&page=81270",
                 "http://wz.sun0769.com/index.php/question/questionType?type=4&page=81330",
                 "http://wz.sun0769.com/index.php/question/questionType?type=4&page=81300"]:
        process = multiprocessing.Process(target=go, args=(url, queue))
        process.start()
        processlist.append(process)  # 开启多个进程

    for p in processlist:
        p.join()  # 等待所有进程退出

    while not queue.empty():
        data = queue.get()
        print("get", data)

In [2]:
# coding:utf-8
# 在jupyter里会报错，但在IDE比如pycharm里面能正常运行并不会报错
import multiprocessing
from multiprocessing import Manager, Process
import time
import requests
import lxml
import re
import lxml.etree
import os
#  输入url,返回一个数据集合list
def  download(url):
    pagetxt = requests.get(url).content  # text返回unicode ,content  str
    # print pagetxt.decode("GB2312",errors="ignore") #必须解码
    myxml = lxml.etree.HTML(pagetxt.decode("GB2312", errors="ignore"))  # 编码
    # 浏览器xpath  mytable=myxml.xpath("//*[@id=\"morelist\"]/div/table[2]/tbody/tr/td/table")
    mytable = myxml.xpath("//*[@cellpadding=\"0\"]//*[@cellpadding=\"1\"]")
    idlist = []
    typelist = []
    titlelist = []
    aboutlist = []
    statuslist = []
    namelist = []
    datalist = []
    for line in mytable:
        idlist = line.xpath("//td[1]/text()")
        typelist = line.xpath("//td[2]/a[1]/text()")
        titlelist = line.xpath("//td[2]/a[2]/text()")
        aboutlist = line.xpath("//td[2]/a[3]/text()")
        statuslist = line.xpath("//td[3]/span/text()")
        namelist = line.xpath("//td[4]/text()")
        datelist = line.xpath("//td[5]/text()")
        for i in range(len(typelist)):
            #print idlist[i + 1], typelist[i], titlelist[i], aboutlist[i], statuslist[i], namelist[i + 1], datelist[
                #i + 1]
            mygetstr = ""
            mygetstr += idlist[i + 1]
            mygetstr += " # "
            mygetstr += typelist[i]
            mygetstr += " # "
            mygetstr += titlelist[i]
            mygetstr += " # "
            mygetstr += aboutlist[i]
            mygetstr += " # "
            mygetstr += statuslist[i]
            mygetstr += " # "
            mygetstr += namelist[i+1]
            mygetstr += " # "
            mygetstr += datelist[i+1]
            mygetstr += "\r\n"  #换行
            datalist.append(mygetstr)
    return  datalist

def geturlnumbers(url):
    pagetxt = requests.get(url).content  # text返回unicode ,content  str
    #print pagetxt.decode("GB2312",errors="ignore") #必须解码
    myxml = lxml.etree.HTML(pagetxt.decode("GB2312",errors="ignore")) #编码
    mylist = myxml.xpath("//*[@class=\"pagination\"]/text()") #抓取数量
    text = mylist[len(mylist) - 1].strip()  #正则表达式提取
    pat = re.compile("\d+",re.IGNORECASE)
    datalist = pat.findall(text)
    return eval(datalist[0])  #81389

def  makeurllist(numbers):
    urllist=[]
    if  numbers%30==0:
        for i  in range(numbers//30):
            urllist.append("http://wz.sun0769.com/index.php/question/questionType?type=4&page="+str(i*30))
    else:
        for i  in range(numbers//30+1):
            urllist.append("http://wz.sun0769.com/index.php/question/questionType?type=4&page=" + str(i * 30))
    return urllist

def go(urllist, queue):
    for  url  in urllist:
        #time.sleep(3)
        try:
            linelist = download(url)
            for line in linelist:
                queue.put(line)  # 进程压入数据
        except:
            print("error")
        else:
            print("%s %s" %(os.getpid(), url))  # 提示这个网页被那个进程抓取成功
    print("%s ed data  exit" %os.getpid())#进程退出

def  readdata(queue):
    file = open("multiprocess_spider.txt","wb")
    while not queue.empty():
        try:
            data = queue.get(timeout=15)
            print("get %s" %data)
            file.write(data.encode("utf-8",errors="ignore"))
            file.flush() #实时生效
        except:
            pass
    file.close()

if __name__ == "__main__":
    numbers = geturlnumbers("http://wz.sun0769.com/index.php/question/questionType?type=4&page=81240")
    urllist = makeurllist(numbers)
    xclist = [[], [], [], [], [], [], [], [], [], []]
    N = len(xclist)
    for i in range(len(urllist)):
        xclist[i % N].append(urllist[i])  # 求模切割

    #queue = multiprocessing.Queue()  # 进程之间传递数据,python2不可以，python3,
    #  error
    queue = Manager().Queue() #多进程
    processlist = []
    for urllist in xclist:
        process = Process(target=go, args=(urllist, queue))
        process.start()
        processlist.append(process)  # 开启多个进程

    readprocess = Process(target=readdata, args=(queue,))#开启读取
    readprocess.start()
    processlist.append(readprocess)

    for p in processlist:
        p.join()  # 等待所有进程退出
    print("ok")

BrokenPipeError: [Errno 32] Broken pipe